In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\user\\Desktop\\deep-learning'

In [4]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_file_path:Path




In [5]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories

In [6]:
import os
os.path.dirname("artifacts/prepare_callbacks/checkpoint_dir/model.h5")

'artifacts/prepare_callbacks/checkpoint_dir'

In [11]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback(self)->PrepareCallbackConfig:
        config=self.config.prepare_callbacks

        model_ckpt_dir=os.path.dirname(config.checkpoint_model_file_path)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config=PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path=Path(config.checkpoint_model_file_path)


        )
        return prepare_callback_config

    

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class PrepareCallback:

    def __init__(self,config:PrepareCallbackConfig):
        self.config=config

    @property
    def create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir,
                                        f"tb_logs_at_{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_file_path,
            save_best_only=True
        )
    
    @property

    def get_tb_callback(self):
        return [
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
        ]


In [14]:
try:
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callback()
    prepare_callback=PrepareCallback(prepare_callback_config)
    callback_list=prepare_callback.get_tb_callback

except Exception as e:
    raise e


2025-06-03 12:29:58,762 - INFO - config\config.yaml loaded successfully
2025-06-03 12:29:58,770 - INFO - params.yaml loaded successfully
2025-06-03 12:29:58,770 - INFO - Created directory artifacts
2025-06-03 12:29:58,778 - INFO - Created directory artifacts\prepare_callbacks\checkpoint_dir
2025-06-03 12:29:58,787 - INFO - Created directory artifacts\prepare_callbacks\tensorboard_log_dir
